In [2]:
%pip install datasets

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 KB 8.4 MB/s eta 0:00:003.7 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 KB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 KB 44.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 13.5 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 KB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 KB 18.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
timm 0.9.0 requires torch>=1.7, which is not installed.
timm 0.9.0 requires torchvision, which is not installed.
open-clip-torch 2.0.2 requires torch>=1.9, which is not installed.
open-clip-torch 2.0.2 requires torchvisio

In [5]:
%pip install torch

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 2.3 MB/s eta 0:00:00m eta 0:00:010:00:03
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.5 MB/s eta 0:00:00m eta 0:00:010:00:02
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 12.7 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 4.5 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 10.5 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 12.4 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 4.5 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.2/176.2 MB 7.4 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 KB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [6]:
import pandas as pd
from datasets import Dataset
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback
from transformers.trainer_callback import TrainerCallback
from sklearn.model_selection import train_test_split
import time

# Load data
data_path = '../dataset/emotions.csv'  # Update this path to your actual file location
df = pd.read_csv(data_path)

# Convert labels to numeric if they're categorical
label_mapping = {label: idx for idx, label in enumerate(df['label'].unique())}
df['label'] = df['label'].map(label_mapping)

# Load data into Hugging Face dataset
dataset = Dataset.from_pandas(df)

# Tokenization
tokenizer = BertTokenizer.from_pretrained('google/bert_uncased_L-4_H-512_A-8')
def tokenize_function(examples):
    return tokenizer(examples['text'], truncation=True, padding="max_length", max_length=128)

tokenized_dataset = dataset.map(tokenize_function, batched=True)

# Setting up training and testing split
train_dataset, test_dataset = tokenized_dataset.train_test_split(test_size=0.1).values()

# Load a smaller BERT variant for sequence classification
model = BertForSequenceClassification.from_pretrained('google/bert_uncased_L-4_H-512_A-8', num_labels=len(label_mapping))
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=5000,
    evaluation_strategy="epoch",
    save_strategy="epoch"
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer
)

# Train the model
trainer.train()

# Evaluate the model
results = trainer.evaluate()
print(results)

Map: 100%|██████████| 416809/416809 [02:31<00:00, 2747.09 examples/s]
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/bert_uncased_L-4_H-512_A-8 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/home/nzappia/.local/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None)
  warnings.warn(
/home/nzappia/.local/lib/python3.10/site-packages/torch/nn/parallel/data_parallel.py:33: UserWarning: 
    There is an imbalance between your GPUs. You may want to exclude GPU 0 which
    has less than 75% of the memory or cores of G

{'loss': 0.289, 'learning_rate': 4.67782582548183e-05, 'epoch': 0.21}


                                                    
  2%|▏         | 1755/70338 [11:56<52:44, 21.67it/s]

{'loss': 0.1322, 'learning_rate': 4.31985452046164e-05, 'epoch': 0.43}


                                                    
  2%|▏         | 1755/70338 [15:30<52:44, 21.67it/s] 

...

{'eval_loss': 0.0901712030172348, 'eval_runtime': 47.6748, 'eval_samples_per_second': 874.277, 'eval_steps_per_second': 54.662, 'epoch': 3.0}


                                                    
100%|██████████| 70338/70338 [55:00<00:00, 21.31it/s]
/home/nzappia/.local/lib/python3.10/site-packages/torch/nn/parallel/data_parallel.py:33: UserWarning: 
    There is an imbalance between your GPUs. You may want to exclude GPU 0 which
    has less than 75% of the memory or cores of GPU 1. You can do so by setting
    the device_ids argument to DataParallel, or by setting the CUDA_VISIBLE_DEVICES
    environment variable.
  warnings.warn(imbalance_warn.format(device_ids[min_pos], device_ids[max_pos]))
/home/nzappia/.local/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


{'train_runtime': 3300.519, 'train_samples_per_second': 340.972, 'train_steps_per_second': 21.311, 'train_loss': 0.11331809622911479, 'epoch': 3.0}


100%|██████████| 2606/2606 [00:46<00:00, 55.48it/s]

{'eval_loss': 0.0901712030172348, 'eval_runtime': 46.9942, 'eval_samples_per_second': 886.94, 'eval_steps_per_second': 55.454, 'epoch': 3.0}


In [7]:
output_dir = "./Bert_Model_Weights"
trainer.save_model(output_dir)

In [16]:
print(results)

{'eval_loss': 0.0901712030172348, 'eval_runtime': 50.1057, 'eval_samples_per_second': 831.861, 'eval_steps_per_second': 52.01, 'epoch': 3.0}


In [17]:
# Use the trained model to generate predictions
predictions = trainer.predict(test_dataset=test_dataset)

# Extract predictions and true labels
predicted_labels = predictions.predictions.argmax(axis=1)
true_labels = test_dataset["label"]


/home/nzappia/.local/lib/python3.10/site-packages/torch/nn/parallel/data_parallel.py:33: UserWarning: 
    There is an imbalance between your GPUs. You may want to exclude GPU 0 which
    has less than 75% of the memory or cores of GPU 1. You can do so by setting
    the device_ids argument to DataParallel, or by setting the CUDA_VISIBLE_DEVICES
    environment variable.
  warnings.warn(imbalance_warn.format(device_ids[min_pos], device_ids[max_pos]))
/home/nzappia/.local/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
100%|██████████| 2606/2606 [00:50<00:00, 51.36it/s]


In [21]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Loss
loss = results["eval_loss"]

# Compute accuracy
accuracy = accuracy_score(true_labels, predicted_labels)

# Compute precision
precision = precision_score(true_labels, predicted_labels, average='macro')

# Compute recall
recall = recall_score(true_labels, predicted_labels, average='macro')

# Compute F1 score
f1 = f1_score(true_labels, predicted_labels, average='macro')

# Print metrics
print(f"Loss: {loss}")
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

Loss: 0.0901712030172348
Accuracy: 0.9415561046999832
Precision: 0.9252082750625599
Recall: 0.9008178959652362
F1 Score: 0.9029663862840299


In [18]:
from sklearn.metrics import classification_report

# Compute classification report
classification_rep = classification_report(true_labels, predicted_labels)

# Print classification report
print("Classification Report:")
print(classification_rep)

Classification Report:
              precision    recall  f1-score   support

           0       0.90      0.90      0.90      4769
           1       0.98      0.98      0.98     12157
           2       0.78      0.99      0.87      3404
           3       0.99      0.93      0.96     14192
           4       1.00      0.62      0.76      1490
           5       0.91      0.99      0.95      5669

    accuracy                           0.94     41681
   macro avg       0.93      0.90      0.90     41681
weighted avg       0.95      0.94      0.94     41681



In [8]:
from sklearn.metrics import classification_report

# Extract predictions and labels
predictions = results["predictions"]
true_labels = results["label_ids"]

# Apply softmax to get probabilities
probabilities = torch.softmax(torch.tensor(predictions), dim=-1)
predicted_labels = torch.argmax(probabilities, dim=-1).tolist()

# Compute classification report
classification_rep = classification_report(true_labels, predicted_labels)

# Print classification report
print("Classification Report:")
print(classification_rep)

KeyError: 'predictions'

In [6]:
%pip install accelerate -U

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
